Text analysis for Lego website-> Anaylsis scrapes lego site and parses through description text to qualify if a set has "minidolls" vs "minifigures". In 2021 Lego took removed it catergorization of "girls" from it's theme list. However there still remains a divide in sets targeted towards girls vs boys. Lego Friends a theme that was first instant of Lego's "minidolls" was created and marketed as Lego for girls. Despite removing the separation on their site Lego continues to make sets with these different figures.

In [149]:
import base64
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import json
from urllib.parse import urljoin
import pandas as pd
import time
from rich import print
from rich.progress import track
from random import randrange

In [6]:
def get_url(
    url: str,
    user_agent: str = "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
    timeout: int = 30,
    verbose: bool = False,
) -> str:
    """Get the provided URL.

    Args:
        url (str): The URL to get.
        user_agent (str, optional): The user agent to use. Defaults to "Mozilla/5.0 (Macintosh; Intel Mac OS X 12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/
        timeout (int, optional): The number of seconds to wait before timing out. Defaults to 30.
        verbose (bool, optional): Whether to print out the results. Defaults to False.

    Returns:
        requests.Response: The response object.
    """
    # Convert the URL to an encoded file name for our cache
    cache_name = base64.b64encode(url.encode("ascii")).decode("utf")

    # Name the cache path
    cache_path = Path("./cache/") / f"{cache_name}.txt"

    # If the cache path exists, just return what's in it:
    if cache_path.exists():
        data = open(cache_path, "r").read()
        if verbose:
            print(f"Pulling {url} from {cache_path}")
        return data
    
    # Set the headers
    headers = {
        "User-Agent": user_agent,
    }

    # Log the URL, if verbose
    if verbose:
        print(f"Fetching {url}")

    # Get the URL
    r = requests.get(url, headers=headers, timeout=timeout)

    # Raise an error if the request failed
    assert r.ok

    # Write the file's contents out to the cache
    with open(cache_path, "w") as cache_file:
        cache_file.write(r.text)

    # Return the response
    return r.text

In [8]:
# get the url
url = "https://www.lego.com/en-us/categories/all-sets?sort.key=NAME&sort.direction=ASC"

r = get_url(url, verbose=True)

soup = BeautifulSoup(r, "html.parser")

Pulling https://www.lego.com/en-us/categories/all-sets?sort.key=NAME&sort.direction=ASC from 
cache/aHR0cHM6Ly93d3cubGVnby5jb20vZW4tdXMvY2F0ZWdvcmllcy9hbGwtc2V0cz9zb3J0LmtleT1OQU1FJnNvcnQuZGlyZWN0aW9uPUFTQw==.
txt

In [9]:
def timer(start_time,end_time):
    """
    This is a function for a timer
    para1 = int for start time
    para2 = int for end time
    """
    snoozer = randrange(start_time,end_time)
    time.sleep(snoozer)

In [10]:
# base url
base_url = "https://www.lego.com"

In [13]:
# number of pages
page_number= 96

In [14]:
#number of min items to expect
number_of_items=  1717

In [15]:
# create a list of urls for each page
urls_links = []
# loop through urls and download them
for i in track(range(1, page_number+1), description="Getting URLs"):
    url = f"https://www.lego.com/en-us/categories/all-sets?page={i}&sort.key=NAME&sort.direction=ASC"
    r = get_url(url, verbose=False)
    soup = BeautifulSoup(r, "html.parser")
    for a in soup.find_all('a', href=True):
        if a['href'].startswith('/en-us/product/'):
            if (base_url + a['href']) not in urls_links:
                urls_links.append(base_url + a['href'])
    timer(1,5)

Output()

In [16]:
# store urls in csv
pd.DataFrame(urls_links).to_csv("./links.csv", index=False)

In [33]:
# read csv
df = pd.read_csv("./links.csv")

In [34]:
# get the urls from the csv file
urls_links = list(df['0'])

In [35]:
# loop through the urls and get the product code, name and featuresText
for url in track(urls_links, description="Getting Data"):
    text = get_url(url, verbose=True)
    soup = BeautifulSoup(text, "html.parser")
    script_txt = soup.find("script",{"id":"__NEXT_DATA__"})
    script=json.loads(script_txt.get_text())
    script.keys()
    data=script['props']['pageProps']['__APOLLO_STATE__']
    for k,v in data.items():
        if "featuresText" in v.keys():
            d = {"productCode":v['productCode'],"name":v['name'],'featuresText':v['featuresText']}
            # create a json file and check if product code is in the json file if not save to json file
            try:
                with open("lego.json", "r") as f:                    existing_data = json.load(f)
            except (FileNotFoundError, json.JSONDecodeError):
                existing_data = []

            if d['productCode'] not in [item['productCode'] for item in existing_data]:
                existing_data.append(d)
                with open("lego.json", "w") as f:
                    json.dump(existing_data, f, indent=4)
                # add timer delay
                timer(1,5)

Output()

Pulling https://www.lego.com/en-us/product/mosaic-maker-40179 from 
cache/aHR0cHM6Ly93d3cubGVnby5jb20vZW4tdXMvcHJvZHVjdC9tb3NhaWMtbWFrZXItNDAxNzk=.txt

Pulling 
https://www.lego.com/en-us/product/christmas-picture-frame-40702?icmp=HP-SHGB-Standard-NT_SWB_40702_GWP_Product_HP-
EX-NO-CWJWPMFM9D from 
cache/aHR0cHM6Ly93d3cubGVnby5jb20vZW4tdXMvcHJvZHVjdC9jaHJpc3RtYXMtcGljdHVyZS1mcmFtZS00MDcwMj9pY21wPUhQLVNIR0ItU3Rhb
mRhcmQtTlRfU1dCXzQwNzAyX0dXUF9Qcm9kdWN0X0hQLUVYLU5PLUNXSldQTUZNOUQ=.txt

Pulling https://www.lego.com/en-us/product/the-botanical-garden-21353 from 
cache/aHR0cHM6Ly93d3cubGVnby5jb20vZW4tdXMvcHJvZHVjdC90aGUtYm90YW5pY2FsLWdhcmRlbi0yMTM1Mw==.txt

Pulling https://www.lego.com/en-us/product/the-endurance-10335 from 
cache/aHR0cHM6Ly93d3cubGVnby5jb20vZW4tdXMvcHJvZHVjdC90aGUtZW5kdXJhbmNlLTEwMzM1.txt

...

Fetching https://www.lego.com/en-us/product/tropical-ukulele-31156

Fetching https://www.lego.com/en-us/product/visit-to-the-vet-clinic-10438

Fetching https://www.lego.com/en-us/product/lloyds-elemental-power-mech-71817

Fetching https://www.lego.com/en-us/product/advent-calendar-2024-42637

Fetching https://www.lego.com/en-us/product/super-nature-5006851

In [36]:
# read json and save to df
df = pd.read_json("lego.json")

In [37]:
# call it
df

,productCode,name,featuresText
0,40179,Mosaic Maker,Creative people will love turning their favori...
1,40702,Christmas Picture Frame,Capture the magic of the holiday season with t...
2,40728,Brite Bomber,Thrill gamers with this awesome LEGO® Fortnite...
3,21353,The Botanical Garden,<p>Build your connection to the natural world ...
4,10335,The Endurance,<p>Indulge in a mindful project or treat a lov...
...,...,...,...
1704,31156,Tropical Ukulele,"<p>Kids can build, play with and display 3 bea..."
1705,10438,Visit to the Vet Clinic,<p>Know a caring toddler who loves animals? Tr...
1706,71817,Lloyd's Elemental Power Mech,<p>Boys and girls aged 7 and up can recreate s...
1707,42637,LEGO® Friends Advent Calendar 2024,<p>Give kids aged 6+ a festive treat as they o...


In [123]:
# create a theme column based on the product code taken from Lego site
df['theme'] = df['productCode'].apply(lambda x: "Star Wars" if str(x).startswith("753") else
                                        "Super Mario" if str(x).startswith("714") else
                                        "Wendnesday" if str(x).startswith("767") else
                                        "Wicked" if str(x).startswith("756") else
                                        "Technic" if str(x).startswith("422") else
                                        "Legend of Zelda" if str(x).startswith("77092") else
                                        "Icons" if str(x).startswith("1033") else
                                        "Ideas" if str(x).startswith("213") else
                                        "Indiana Jones" if str(x).startswith("7701") else
                                        "Animal Crossing" if str(x).startswith("770") else
                                        "Architecture" if str(x).startswith("210") else
                                        "Art" if str(x).startswith("312") else
                                        "Superheroes" if str(x).startswith("762") else
                                        "DC" if str(x).startswith("76302") else
                                        "Despicable Me" if str(x).startswith("7558") else
                                        "Monkie Kid" if str(x).startswith("800") else
                                        "Ninjago" if str(x).startswith("718") else
                                        "Sonic" if str(x).startswith("7699") else
                                        "Speed Champions" if str(x).startswith("7724") else
                                        "Minecraft" if str(x).startswith("212") else
                                        "Disney" if str(x).startswith("432") else
                                        "Dreamz" if str(x).startswith("7148") else
                                        "Duplo" if str(x).startswith("109") else
                                        "Peppa Pig" if str(x).startswith("1043") else
                                        "Friends" if str(x).startswith("426") else
                                        "Fornite" if str(x).startswith("7707") else
                                        "4 plus" if str(x).startswith("107") else
                                        "Harry Potter" if str(x).startswith("764") else
                                        "other" if str(x).startswith("407") else
                                        "City" if str(x).startswith("604") else
                                        "Classic" if str(x).startswith("110") else
                                        "Botanical Collection" if str(x).startswith("103") else
                                        "Creator 3-in-1" if str(x).startswith("311") else
                                        "Avatar" if str(x).startswith("7557") else
                                        "Unknown")



In [124]:
# call it
df

,productCode,name,featuresText,minidoll,minifigure,theme
0,40179,Mosaic Maker,Creative people will love turning their favori...,False,False,Unknown
1,40702,Christmas Picture Frame,Capture the magic of the holiday season with t...,False,False,other
2,40728,Brite Bomber,Thrill gamers with this awesome LEGO® Fortnite...,False,False,other
3,21353,The Botanical Garden,<p>Build your connection to the natural world ...,False,True,Ideas
4,10335,The Endurance,<p>Indulge in a mindful project or treat a lov...,False,False,Icons
...,...,...,...,...,...,...
1704,31156,Tropical Ukulele,"<p>Kids can build, play with and display 3 bea...",False,False,Creator 3-in-1
1705,10438,Visit to the Vet Clinic,<p>Know a caring toddler who loves animals? Tr...,False,False,Peppa Pig
1706,71817,Lloyd's Elemental Power Mech,<p>Boys and girls aged 7 and up can recreate s...,False,True,Ninjago
1707,42637,LEGO® Friends Advent Calendar 2024,<p>Give kids aged 6+ a festive treat as they o...,False,False,Friends


In [125]:
# do a quick check
df.sample(15)

,productCode,name,featuresText,minidoll,minifigure,theme
268,75405,Home One Starcruiser,<p>Inspire nostalgia for Star Wars: Return of ...,False,False,Unknown
355,854187,Grogu™ Key Chain,Star Wars: The Mandalorian fans can display th...,False,True,Unknown
1624,853990,Easter Bunny House,Celebrate Easter with this fun LEGO® Iconic Ea...,False,True,Unknown
1415,5005892,Minifigure Lunch Set,<p>Make LEGO® fans smile every lunchtime with ...,False,True,Unknown
365,31152,Space Astronaut,<p>LEGO® fans aged 9+ can blast into space to ...,False,True,Creator 3-in-1
244,40725,Cherry Blossoms,<p>Mark the arrival of spring and new beginnin...,False,False,other
1433,5005353,LEGO® Red Cinch Bucket,<p>Store your gear tidily and transport it in ...,False,False,Unknown
231,10454,Mickey Mouse Clubhouse & Car,<p>Immerse toddlers in the fun of their favori...,False,False,Unknown
1035,5007111,Wooden Desk Drawer 8 – Light Oak,"LEGO® fans will love this fun, practical and a...",False,False,Unknown
1354,5007108,Wooden Book Rack – Dark Oak,This versatile rack provides the perfect place...,False,False,Unknown


In [126]:
# look through the featuresText column for "minidoll" and "minifigure"
# create column that says if text does or doesn't contain minidoll or mini figure
df['minidoll'] = df['featuresText'].apply(lambda x: "minidoll" in x.lower() or "mini-doll" in x.lower())
df['minifigure'] = df['featuresText'].apply(lambda x: "minifigure" in x.lower())



In [129]:
# check it 
df.sample(45)

,productCode,name,featuresText,minidoll,minifigure,theme
595,10969,Fire Truck,<p>Kids aged 2+ with a fascination for fire tr...,False,False,Duplo
1599,40409,Hot Rod,Discover a cool replica of the much-loved LEGO...,False,True,Unknown
358,42182,NASA Apollo Lunar Roving Vehicle - LRV,<p>Embark on a journey to the Moon with this d...,False,False,Unknown
138,75382,TIE Interceptor™,<p>Spark memories of classic Star Wars™ saga a...,False,True,Star Wars
72,76314,Captain America: Civil War Action Battle,<p>Captain America: Civil War Action Battle (7...,False,True,Unknown
301,31160,Aircraft: Race Plane,<p>Superfast adventures in the sky await boys ...,False,False,Creator 3-in-1
731,21271,The Trial Chamber,<p>The Trial Chamber (21271) is a high-quality...,False,True,Minecraft
464,42168,John Deere 9700 Forage Harvester,<p>Kids who love farm equipment toys can enjoy...,False,False,Unknown
1569,5006591,"Storage Head – Small, Skeleton",LEGO® fans will love keeping small items organ...,False,True,Unknown
860,10320,Eldorado Fortress,<p>Rediscover the world of LEGO® Pirates with ...,False,True,Botanical Collection


In [151]:
with_minidolls = df[df['minidoll']==True]['theme'].value_counts()
with_minidolls

theme
Friends       51
Disney        33
Unknown       26
4 plus         7
Wicked         3
other          2
Wendnesday     1
Name: count, dtype: int64

In [153]:
# count the number of themes with no minidolls but have minifigures
with_minifigures =df[(df['minidoll']==False) & (df['minifigure']==True)]['theme'].value_counts()
with_minifigures

theme
Unknown                 384
City                     60
Star Wars                51
Superheroes              42
Ninjago                  34
Super Mario              29
Harry Potter             29
Monkie Kid               20
Botanical Collection     17
Ideas                    16
Animal Crossing          15
Disney                    9
Speed Champions           8
Creator 3-in-1            8
other                     6
4 plus                    6
Icons                     4
Avatar                    4
Minecraft                 3
Indiana Jones             3
Sonic                     2
DC                        1
Legend of Zelda           1
Despicable Me             1
Name: count, dtype: int64

In [154]:

# count the number of themes with no minidolls and no minifigures
without_both = df[(df['minidoll']==False) & (df['minifigure']==False)]['theme'].value_counts()
without_both

theme
Unknown                 533
Duplo                    35
Creator 3-in-1           33
Minecraft                29
Super Mario              26
Botanical Collection     23
Classic                  21
Superheroes              19
other                    18
Star Wars                12
Ideas                    11
Disney                    9
Architecture              9
Technic                   8
Sonic                     8
Harry Potter              8
Art                       7
Peppa Pig                 7
Icons                     5
Animal Crossing           4
Despicable Me             3
Friends                   2
4 plus                    1
Wicked                    1
Wendnesday                1
Name: count, dtype: int64

In [155]:
# number of sets in each theme
sets = df['theme'].value_counts()
sets

theme
Unknown                 943
Star Wars                63
Superheroes              61
City                     60
Super Mario              55
Friends                  53
Disney                   51
Creator 3-in-1           41
Botanical Collection     40
Harry Potter             37
Duplo                    35
Ninjago                  34
Minecraft                32
Ideas                    27
other                    26
Classic                  21
Monkie Kid               20
Animal Crossing          19
4 plus                   14
Sonic                    10
Architecture              9
Icons                     9
Technic                   8
Speed Champions           8
Art                       7
Peppa Pig                 7
Avatar                    4
Despicable Me             4
Wicked                    4
Indiana Jones             3
Wendnesday                2
Legend of Zelda           1
DC                        1
Name: count, dtype: int64

In [146]:
# number of minidoll sets in superheroes
df[(df['theme']=="Superheroes") & (df['minidoll']==True)]['theme'].value_counts()

Series([], Name: count, dtype: int64)

In [147]:
# number of minidoll sets in City
df[(df['theme']=="City") & (df['minidoll']==True)]['theme'].value_counts()

Series([], Name: count, dtype: int64)

In [148]:
# number of minidoll sets in friends
df[(df['theme']=="Friends") & (df['minidoll']==True)]['theme'].value_counts()

theme
Friends    51
Name: count, dtype: int64

In [156]:
# save the data to Excel with each variable in a separate page of Excel file
with pd.ExcelWriter('lego_data.xlsx') as writer:
    with_minidolls.to_excel(writer, sheet_name='with_minidolls')
    with_minifigures.to_excel(writer, sheet_name='with_minifigures')
    without_both.to_excel(writer, sheet_name='without_both')
    sets.to_excel(writer, sheet_name='sets')
    df.to_excel(writer, sheet_name='data')
